In [1]:
from mpi import create_distinct_view, create_identity_view, link
import pandas as pd
import os
from utils.db import get_session
from utils.models import *

## Ingest
Bring .csv into the system

In [2]:
from ingest import load_file
os.listdir('assets/data')

['dws_wages.csv',
 'usbe_students.csv',
 'ushe_students.csv',
 'ustc_students.csv']

In [3]:
load_file('assets/data/dws_wages.csv', 'dws_wages')
load_file('assets/data/usbe_students.csv', 'usbe_students')
load_file('assets/data/ustc_students.csv', 'ustc_students')
load_file('assets/data/ushe_students.csv', 'ushe_students')

## Linking Steps

Process of Record Linkage

0. Load
1. Preprocess
2. Index
3. Compare
4. Classify
5. Evaluate
6. Update (Post MPI - Adding new information, new MPI, to MPI Master Record)

### Master Person Long
Visualize data architecture store of MPI raw records

In [4]:
# Visualize raw MPI data
#  V0.1  Data is seeded with ushe_students table
#  TODO: implement matched route and proper identification of unmatched

with get_session() as session:
    result = session.execute(
        'SELECT * FROM master_person_long LIMIT 5'
    ).fetchall()
    count_mpi = session.execute(
        'SELECT COUNT(DISTINCT(mpi)) FROM master_person_long'
    ).fetchone()
columns = ('mpi', 'field', 'value', 'score', 'guid')
pd.DataFrame(result, columns=columns)

,mpi,field,value,score,guid
0,4587782-4309556-16453842-9029444,ushe_student_id_pool,469629.0,1.0,1595745416251645427
1,4587782-4309556-16453842-9029444,last_name_pool,Steed,1.0,1595745416251645427
2,4587782-4309556-16453842-9029444,first_name_pool,Sakayla,1.0,1595745416251645427
3,4587782-4309556-16453842-9029444,middle_name_pool,Conder,1.0,1595745416251645427
4,4587782-4309556-16453842-9029444,birth_date_pool,6/20/2859,1.0,1595745416251645427


In [5]:
print('Total MPI in system: ', count_mpi[0])

Total MPI in system:  29330


## Load Data

In [6]:
source_tablename = 'dws_wages'

In [7]:
# Create a view of the data with mapped columns
raw, subset = create_distinct_view(source_tablename)
dview = subset.drop_duplicates()
# print(dview.head())

# Create a view from the MPI table with valid identity data
iview = create_identity_view(dview.columns.tolist())
# print(iview.head())

In [8]:
dview.head(1)

,ssn_pool,last_name_pool,first_name_pool,middle_name_pool,guid
0,885713242,Neddo,Giniveve Dee,Sky Marie,1627838511399422658


In [9]:
iview.head(1)

value                       \
field                            first_name_pool                 guid   
mpi                                                                     
10000079-6468912-4595469-3226763        Maryalie  1834271269758541656   

                                                                             \
field                            last_name_pool middle_name_pool   ssn_pool   
mpi                                                                           
10000079-6468912-4595469-3226763      Sarmiento  Barrett Bradley  766178317   

                                           score                      \
field                            first_name_pool guid last_name_pool   
mpi                                                                    
10000079-6468912-4595469-3226763               1    1              1   

                                                            
field                            middle_name_pool ssn_pool  
mpi                                                         
10000079-6468912-4595469-3226763                1        1

Check here for potential for match.  If NO matching columns exist, mpis will need to be created for the dview.

In [10]:
# Check for match availability.  If not, halt process and create MPIs
from mpi.link import is_match_available
from mpi.update import generate_mpi

if not is_match_available(dview, iview):
    generate_mpi(dview)
    print('Match unavailable.  Generated MPIs for data view.')
    # Recreate a view from the MPI table with valid identity data
    iview = create_identity_view(dview.columns.tolist())
else:
    print('Match available.  Proceed with linking process.')

Match available.  Proceed with linking process.


## Building record linkage and mpi classification

In [11]:
from mpi.link import clean_raw, match_dtype

### Preprocessing

Standardize data across data and identity views.

In [12]:
# Match Dtypes - Align data types prior to cleaning.
#    This helps the cleaner by segmenting string/object and numeric fields

# t2 identity table is a multi-index table.
#    If the table is empty, it will not have the value & score attributes
#    The if/else just checks whether or not the pivot of the mpi view yielded value and score fields.

if hasattr(iview, 'value'):
    ivalue = iview.value
else:
    ivalue = iview
    
if hasattr(iview, 'score'):
    iscore = iview.score

# Cast columns to matching datatypes for comparisons later on
source_data, id_data = match_dtype(dview, ivalue)  

# Clean data and re-index comparison.
subset = clean_raw(subset)
source_data = clean_raw(source_data)
id_data = clean_raw(id_data)
id_data = id_data.reset_index(level='mpi')

In [13]:
source_data.head(1)

,ssn_pool,last_name_pool,first_name_pool,middle_name_pool,guid
0,885713242,neddo,giniveve dee,sky marie,1627838511399422658


In [14]:
id_data.head(1)

field,mpi,first_name_pool,guid,last_name_pool,middle_name_pool,ssn_pool
0,10000079-6468912-4595469-3226763,maryalie,1834271269758541656,sarmiento,barrett bradley,766178317


In [15]:
iscore.head(1)

field,first_name_pool,guid,last_name_pool,middle_name_pool,ssn_pool
mpi,,,,,
10000079-6468912-4595469-3226763,1,1,1,1,1


## Indexing

Make record pairs - pair rows needing match to potential identity candidates.

Indexing serves two purposes:

1. Create the list of pairs to check (candidate link).  Example: row 1 from table 1 to row 199 from table 2.

2. Reduce the potential number of pairs to check (candidates).

In [16]:
from mpi.link import build_indexer, match_columns

In [17]:
# Create indexer on dataview
#    Indexer is a set of rules to generate 
#    candidate matches from data -> identities

source_matched, id_matched = match_columns(source_data, id_data)

indexer = build_indexer(source_matched)

# Check index algorithms (generated from data view columns)
indexer.algorithms

[<SortedNeighbourhood left_on='last_name_pool', right_on='last_name_pool'>,
 <SortedNeighbourhood left_on='first_name_pool', right_on='first_name_pool'>,
 <SortedNeighbourhood left_on='middle_name_pool', right_on='middle_name_pool'>,
 <Block left_on='ssn_pool', right_on='ssn_pool'>]

In [18]:
# Run indexer on dataview, identity view
candidates = indexer.index(source_matched, id_matched)

# Full indexing is a cross join of data and all possible identities.

# Demonstrating full indexing size:
print('Full Index Length: ', len(source_data) * len(id_data))

# Examine multi indices.  On the left is the data view index.  Right identity.
print('Algorithmic Index Length: ', len(candidates))

# Estimate Savings
print('Savings: ', (1- len(candidates)/(len(source_data) * len(id_data))) * 100)

# Preview indices:
for pair in candidates[0:5]:
    print(f'Data-row {pair[0]}', f'ID-row {pair[1]}')

Full Index Length:  293300000
Algorithmic Index Length:  150206
Savings:  99.9487875894988
Data-row 0 ID-row 741
Data-row 0 ID-row 4476
Data-row 0 ID-row 6621
Data-row 0 ID-row 6767
Data-row 0 ID-row 8705


## Comparing

Indexing does not normally store the outcome of its findings.  Indexing algorithms are meant to be fast, can be error prone.  Algorithms can be tuned for string (many), numeric, and time/date fields.

The output of comparison is a clean feature matrix for the classifier to train/predict on.

In [19]:
from mpi.link import build_comparator

In [20]:
# Create comparator on dataview
#    Comparator is a set of algorithms for each feature to be compared.
#    These are genearlly much more expensive compared to indexing functions
cmp = build_comparator(source_matched)

# Check comparison algorithms and fields
cmp.features

[<Numeric 'ssn_pool'>,
 <String 'last_name_pool'>,
 <String 'first_name_pool'>,
 <String 'middle_name_pool'>]

In [21]:
# Compute comparisons
#    Gives clean match dataset for classification
comparisons = cmp.compute(candidates, source_data, id_data)
comparisons.head()

ssn_pool  last_name_pool  first_name_pool  middle_name_pool
0 741        0.0             0.0              0.0               0.0
  4476       0.0             0.0              0.0               1.0
  6621       0.0             0.0              0.0               0.0
  6767       1.0             1.0              1.0               1.0
  8705       0.0             1.0              0.0               0.0

## Classification

Score candidates for match.  

#### Two approaches: Supervised vs Unsupervised
 * **Supervised** approach requires a training set.
 * **Unsupervised** does not require a training set and operates on only on the comparison table itself.

In [22]:
from mpi.link import estimate_true, build_classifier

# Get estimated true linkages for supervised model
links_true = estimate_true(comparisons)

# Create classifier
clf = build_classifier('logistic', comparisons, match_index=links_true)

# Check probabilities (score) of each comparison -- NOT IN USE IN THIS VERSION
predictions = clf.prob(comparison_vectors=comparisons)
predictions

0     741      5.285092e-07
      4476     3.269493e-05
      6621     5.285092e-07
      6767     9.990896e-01
      8705     4.084696e-05
                   ...     
9999  20403    4.084696e-05
      20504    5.285092e-07
      26387    3.565263e-05
      26808    4.084696e-05
      28735    4.084696e-05
Length: 150206, dtype: float64

## Evaluate
Express classification quality and explore outliers

In [23]:
from recordlinkage import reduction_ratio
from recordlinkage import confusion_matrix

links_pred = clf.predict(comparison_vectors=comparisons)

rratio = reduction_ratio(links_pred, source_data)
cmatrix = confusion_matrix(links_true, links_pred, candidates)

In [24]:
# Review confusion matrix
#    TP-FN
#    |  |
#    FP-TN
print(cmatrix)

# Review reduction ratio
print(rratio)

[[ 10668      0]
 [     1 139537]]
0.999786598659866


The confusion matrix may not be particularly useful here as generation of true links is prone to error. The reduction ratio is more sensitive than binary predictions in this case.

In [25]:
# Review findings
#   Interesting that the logistic predicted an MPI indices for each given an incomplete target list.

# Is the relationship 1,1?
split_list = lambda x: ([ix[0] for ix in x], [ix[1] for ix in x])
i1, i2 = split_list(links_pred)
len(list(set(i1))), len(list(set(i2)))

(10000, 10669)

## Update

Append matched MPIs and match score to data view and merge to original data.

In [26]:
from mpi.link import expand_match_to_raw

# Join data view (DISTINCT identities in source table), now containing matched and generated MPIs, to raw table.
#    This can be done a few ways.  Here, the data view (whose columns have been renamed and processed)
#    is joined to the original subset (whose columns were just renamed).  The subset is then indexed back 
#    unto the raw table so original column names and source formatting are preserved.


raw, matched, unmatched = expand_match_to_raw(raw, subset, source_data, id_data, links_pred)
raw.head(1)

,CLMWAGES,EMPLOYER,ES_WORKSITE_NUM,NAICS,WAGES,YRQTR,SSN,LASTNAME,FIRSTNAME,MIDDLEINITIAL,id,guid,mpi
0,14188,HOLLYWOOD VIDEO,0,873477,7898926,20172,885713242,Neddo,Giniveve Dee,Sky Marie,10465,1627838511399422658,13501288-9218906-9250909-16175082


In [29]:
id_data.head()

field,mpi,first_name_pool,guid,last_name_pool,middle_name_pool,ssn_pool
0,10000079-6468912-4595469-3226763,maryalie,1834271269758541656,sarmiento,barrett bradley,766178317
1,10000598-552386-272793-10739067,lillyahnie,781581245139754840,landon,paza,865337889
2,10000639-11044250-5124768-5031866,delilah,1834271269758541656,chenault,seph,710504138
3,10000700-736429-12469678-3639982,urijah,1595745416251645427,fortney,elizabeth kamalii wahine kai,450048739
4,10001650-1330169-10532014-11768145,kesian,1834271269758541656,finn,lynette louise,434059061


### Adding missing data, repeated data

When MPIs are generated for unmatched records, all data is used to populate the MPI table.  For matched MPIs, an update to missing information now needs to be performed to capture any new data.

What information is populated can have large performance and storage ramifications.
- If repeat data is kept, the initial identity view will need consolidation to reduce the number of possible indexes and comparisons.
- If repead data is NOT kept, significant information for background updates in match quality will be lost.

In [ ]:
# TODO

### De-Identification

Create de-identified table while match available in memory or as referenced temp table.

In [30]:
from assets.mapping import colmap
from utils.db import dataframe_to_db
from di import simple_di

dataframe_to_db(
    simple_di(raw), 
    tablename=source_tablename + '_di'
)

'ushe_students_di'